In [1]:
import numpy as np
import pandas as pd

from pathlib import Path

from src.metric_utils import load_tranco_list_from_custom_path


### CONFIG

In [2]:
latest_month_metrics_fmt = "2024-04"
latest_month_text_fmt = "Apr 2024"

### Load necessary CRAWL data

In [3]:
crawled_deployment = pd.read_parquet(f"../data/crawled_metrics/nel_deployment/{latest_month_metrics_fmt}.parquet")
crawled_deployment

,date,total_crawled_resources,total_crawled_domains,total_crawled_resources_with_nel,total_crawled_domains_with_nel,total_crawled_resources_with_correct_nel,total_crawled_domains_with_correct_nel
0,2024-04,3962754,23183,3943304,23084,3943299,23083


In [4]:
crawled_collector_provider_usage = pd.read_parquet(f"../data/crawled_metrics/nel_collector_provider_usage/{latest_month_metrics_fmt}.parquet")

In [5]:
crawled_config_is = pd.read_parquet(f"../data/crawled_metrics/nel_config/include_subdomains_{latest_month_metrics_fmt}.parquet")

In [6]:
crawled_config_ff = pd.read_parquet(f"../data/crawled_metrics/nel_config/failure_fraction_{latest_month_metrics_fmt}.parquet")

In [7]:
crawled_config_sf = pd.read_parquet(f"../data/crawled_metrics/nel_config/success_fraction_{latest_month_metrics_fmt}.parquet")

In [8]:
crawled_config_ma = pd.read_parquet(f"../data/crawled_metrics/nel_config/max_age_{latest_month_metrics_fmt}.parquet")

In [9]:
crawled_resource_monitoring_stats = pd.read_parquet(f"../data/crawled_metrics/nel_domain_resource_monitoring_stats/{latest_month_metrics_fmt}.parquet")

In [10]:
crawled_monitored_resource_types = pd.read_parquet(f"../data/crawled_metrics/nel_monitored_resource_types/{latest_month_metrics_fmt}.parquet")

In [11]:
crawled_resources_config_variability = pd.read_parquet(f"../data/crawled_metrics/nel_resource_config_variability/{latest_month_metrics_fmt}.parquet")

#### Load TRANCO 

In [12]:
year = latest_month_metrics_fmt.split("-")[0]
month = latest_month_metrics_fmt.split("-")[1]
tranco_list = load_tranco_list_from_custom_path(Path(f"../resources/"), year, month)
tranco_list

,order,popular_domain_name
0,1,google.com
1,2,facebook.com
2,3,a-msedge.net
3,4,amazonaws.com
4,5,microsoft.com
...,...,...
999995,999996,dvd-collection.com
999996,999997,dvdweb.in
999997,999998,dvl666.net
999998,999999,dwa136.com


## Prepare list of actually CRAWLED domains to be compared to HTTP ARCHIVE results 

In [13]:
crawled_domains = pd.read_parquet("../data/domains_to_crawl.parquet")

#### Domains that were intended to be crawled

In [14]:
intended_domains_to_compare = crawled_domains[['url_domain']]
intended_domains_to_compare

,url_domain
10204,bing.com
87160,zoom.us
63076,reddit.com
85063,yandex.ru
20414,discord.com
...,...
21854,drrashelofficial.com.pk
21919,dt5515.com
21931,du4sas3t.xyz
21962,ducadimorrone.com


#### Filter to domains that were actually attempted to be crawled 

In [15]:
crawl_data_raw_path = Path("../data/crawled_raw/blobs")

available_crawled_data_files = list(crawl_data_raw_path.glob('*.parquet'))
available_crawled_domains = list(map(lambda file: file.stem, available_crawled_data_files))
available_crawled_domains = pd.DataFrame({'url_domain': available_crawled_domains})

domains_crawl_attempted = available_crawled_domains[available_crawled_domains['url_domain'].isin(intended_domains_to_compare['url_domain'])]
domains_crawl_attempted

,url_domain
0,0115765.com
1,013info.rs
2,037hd.tv
3,0x00sec.org
4,1-800-phonesex.com
...,...
33065,zyciesokolowa.pl
33066,zygorguides.com
33067,zynca.se
33068,zzapomni.com


#### Filter to domains that were successfully crawled

In [16]:
latest_crawl_raw_data_path = list(Path("../data/crawled_raw/").glob("merged_*.parquet"))[-1]
latest_crawl_raw_data = pd.read_parquet(latest_crawl_raw_data_path)

domains_crawled = pd.DataFrame({'url_domain': latest_crawl_raw_data['url_domain'].unique()})

domains_crawled.size

23083

## RESULTS:

### Deployment stats

In [17]:
total_domains_crawl_size = domains_crawl_attempted.size
total_domains_crawl_size

33070

In [18]:
total_domains_crawl_failed = domains_crawl_attempted.size - domains_crawled.size
total_domains_crawl_failed

9987

In [19]:
crawled_deployment['total_crawled_domains'][0]

23183

In [20]:
crawled_deployment['total_crawled_domains_with_correct_nel'][0]

23083

In [21]:
crawled_deployment['total_crawled_domains_with_correct_nel'] / crawled_deployment['total_crawled_domains'] * 100

0    99.568649
dtype: float64

### Collector Provider Stats

In [22]:
crawled_collector_provider_usage

,date,providers,as_primary,share_as_primary,as_secondary,share_as_secondary,among_fallback
0,2024-04,cloudflare.com,22910.0,99.250531,0.0,0.0,0.0
1,2024-04,heroku.com,129.0,0.558853,0.0,0.0,0.0
2,2024-04,yandex.net,11.0,0.047654,8.0,80.0,0.0
3,2024-04,report-uri.com,10.0,0.043322,0.0,0.0,0.0
4,2024-04,uriports.com,5.0,0.021661,0.0,0.0,0.0
5,2024-04,hhdev.ru,5.0,0.021661,1.0,10.0,0.0
6,2024-04,3gl.net,3.0,0.012997,0.0,0.0,0.0
7,2024-04,gkd-re.de,3.0,0.012997,0.0,0.0,0.0
8,2024-04,wikimedia.org,2.0,0.008664,0.0,0.0,0.0
9,2024-04,csrtech.support,1.0,0.004332,0.0,0.0,0.0


### Configuration stats

In [23]:
crawled_config_is

,date,nel_include_subdomains,domain_count,domain_percent
0,2024-04,false,23061,99.904692
1,2024-04,true,22,0.095308


In [24]:
crawled_config_ff

,date,nel_failure_fraction,domain_count,domain_percent
0,2024-04,0.001,1,0.004332
1,2024-04,0.01,3,0.012997
2,2024-04,0.05,132,0.571849
3,2024-04,0.1,9,0.038990
4,2024-04,0.2,1,0.004332
5,2024-04,0.5,1,0.004332
6,2024-04,1.0,22936,99.363168


In [25]:
crawled_config_sf

,date,nel_success_fraction,domain_count,domain_percent
0,2024-04,0.0,21798,94.433133
1,2024-04,0.001,10,0.043322
2,2024-04,0.005,129,0.558853
3,2024-04,0.01,1145,4.960360
4,2024-04,0.05,1,0.004332


In [26]:
crawled_config_ma

,date,nel_max_age,domain_count,domain_percent
0,2024-04,100,7,0.030325
1,2024-04,600,1,0.004332
2,2024-04,3600,134,0.580514
3,2024-04,7200,3,0.012997
4,2024-04,86400,2,0.008664
5,2024-04,604800,22914,99.267859
6,2024-04,1209600,1,0.004332
7,2024-04,2592000,14,0.060651
8,2024-04,10886400,1,0.004332
9,2024-04,31536000,6,0.025993


### Resource stats

In [27]:
crawled_deployment['total_crawled_resources'][0]

3962754

In [28]:
crawled_resource_monitoring_stats

,date,url_domain,url_domain_hosted_resources,url_domain_hosted_resources_with_nel,url_domain_monitored_resources_ratio
0,2024-04,013info.rs,168,168,100.000000
1,2024-04,037hd.tv,200,200,100.000000
2,2024-04,0x00sec.org,237,237,100.000000
3,2024-04,1-800-phonesex.com,182,182,100.000000
4,2024-04,1-win.casino,105,105,100.000000
...,...,...,...,...,...
23078,2024-04,zwaar.co,67,66,98.507463
23079,2024-04,zwiftinsider.com,345,345,100.000000
23080,2024-04,zwr.gg,142,141,99.295775
23081,2024-04,zycie.news,118,118,100.000000


In [29]:
from results.result_utils import get_first_or_0

distribution_result = pd.DataFrame({}, index=[
    '(0%-10%>',
    '(10%-25%>',
    '(25%-50%>',
    '(50%-75%>',
    '(75%-100%)',
    '<100%>',
])

distr_col = []

data = crawled_resource_monitoring_stats.copy()

# month_result['(0%-10%>']    
next_val = data[(data['url_domain_monitored_resources_ratio'] > 0.0) & (data['url_domain_monitored_resources_ratio'] <= 10.0)].count()
distr_col.append(get_first_or_0(next_val))

# month_result['(10%-25%>']    
next_val = data[(data['url_domain_monitored_resources_ratio'] > 10.0) & (data['url_domain_monitored_resources_ratio'] <= 25.0)].count()
distr_col.append(get_first_or_0(next_val))

# month_result['(25%-50%>']    
next_val = data[(data['url_domain_monitored_resources_ratio'] > 25.0) & (data['url_domain_monitored_resources_ratio'] <= 50.0)].count()
distr_col.append(get_first_or_0(next_val))

# month_result['(50%-75%>']    
next_val = data[(data['url_domain_monitored_resources_ratio'] > 50.0) & (data['url_domain_monitored_resources_ratio'] <= 75.0)].count()
distr_col.append(get_first_or_0(next_val))

# month_result['(75%-100%)']    
next_val = data[(data['url_domain_monitored_resources_ratio'] > 75.0) & (data['url_domain_monitored_resources_ratio'] < 100.0)].count()
distr_col.append(get_first_or_0(next_val))

# month_result['<100%>']
next_val = data[data['url_domain_monitored_resources_ratio'] == 100.0].count()
distr_col.append(get_first_or_0(next_val))


distribution_result[latest_month_text_fmt] = distr_col
distribution_result

,Apr 2024
(0%-10%>,2
(10%-25%>,628
(25%-50%>,38
(50%-75%>,27
(75%-100%),2452
<100%>,19936


In [30]:
distribution_result[distribution_result.index == '<100%>'][latest_month_text_fmt].iloc[0] / crawled_deployment['total_crawled_domains']

0    0.85994
Name: total_crawled_domains, dtype: float64

In [31]:
distribution_result[distribution_result.index == '(75%-100%)'][latest_month_text_fmt].iloc[0] / crawled_deployment['total_crawled_domains']

0    0.105767
Name: total_crawled_domains, dtype: float64

In [32]:
distribution_result[distribution_result.index == '(10%-25%>'][latest_month_text_fmt].iloc[0] / crawled_deployment['total_crawled_domains']

0    0.027089
Name: total_crawled_domains, dtype: float64

#### Config variability

In [33]:
config_variation_count_data = pd.DataFrame()
current_month_data = crawled_resources_config_variability[['date', 'url_domain', 'nel_include_subdomains']].copy()

config_variations_by_url_domain = current_month_data.groupby(['url_domain'])['nel_include_subdomains'].count()
current_month_data['variation_count'] = current_month_data['url_domain'].map(config_variations_by_url_domain)
current_month_data.drop(columns=['nel_include_subdomains'], inplace=True)
    
current_month_data = current_month_data.groupby(['variation_count']).agg(domains=('url_domain', 'count'))
current_month_data.reset_index(inplace=True)    

current_month_data['date'] = latest_month_text_fmt
current_month_data = current_month_data[['date', 'variation_count', 'domains']] 

current_month_data['share'] = current_month_data['domains'] / crawled_deployment['total_crawled_domains_with_correct_nel']

config_variation_count_data = pd.concat([config_variation_count_data, current_month_data])
    
config_variation_count_data

,date,variation_count,domains,share
0,Apr 2024,1,22842,0.989559
1,Apr 2024,2,480,NaN
2,Apr 2024,3,3,NaN


### Resource types

In [34]:
resource_type_data = pd.DataFrame()
current_month_data = crawled_monitored_resource_types.copy()

current_month_data = current_month_data.groupby(['date', 'type'], observed=True, as_index=False).agg({'count': 'sum'})
current_month_data.sort_values(by='count', ascending=False, inplace=True)
current_month_data.reset_index(inplace=True, drop=True)

current_month_data['date'] = latest_month_text_fmt

resource_type_data = pd.concat([resource_type_data, current_month_data])
    
resource_type_data

,date,type,count
0,Apr 2024,image,2457577
1,Apr 2024,script,483698
2,Apr 2024,html,482927
3,Apr 2024,css,312895
4,Apr 2024,other,100116
5,Apr 2024,,47643
6,Apr 2024,font,36835
7,Apr 2024,text,12527
8,Apr 2024,video,5624
9,Apr 2024,audio,2770


In [35]:
# import numpy as np
# domains_to_analyze = config_variations_by_url_domain[config_variations_by_url_domain == 1].copy()
# domains_to_analyze = domains_to_analyze.reset_index()
# data = latest_crawl_raw_data[latest_crawl_raw_data['url_domain'].isin(domains_to_analyze['url_domain'])]
# data = data[['type', 'url', 'url_domain', 'nel_include_subdomains', 'nel_failure_fraction', 'nel_success_fraction', 'nel_max_age', 'nel_report_to', 'rt_collectors']]
# variation_distr = (data.groupby(
#     ['url_domain', 'nel_include_subdomains', 'nel_failure_fraction', 'nel_success_fraction', 'nel_max_age']
#     , as_index=False, observed=True)
#  .agg(
#     types=('type', lambda x: dict(x.value_counts())),
#     n_monitored=('type', 'count'),
#     rt_collectors=('rt_collectors', lambda x: dict(x.explode().value_counts())),
#     urls=('url', np.hstack)
# ))
# variation_distr

In [36]:
import numpy as np

domains_to_analyze = config_variations_by_url_domain[config_variations_by_url_domain > 1].copy()
domains_to_analyze = domains_to_analyze.reset_index()

data = latest_crawl_raw_data[latest_crawl_raw_data['url_domain'].isin(domains_to_analyze['url_domain'])]

data = data[['type', 'url', 'url_domain', 'nel_include_subdomains', 'nel_failure_fraction', 'nel_success_fraction', 'nel_max_age', 'nel_report_to', 'rt_collectors']]
variation_distr = (data.groupby(
    ['url_domain', 'nel_include_subdomains', 'nel_failure_fraction', 'nel_success_fraction', 'nel_max_age']
    , as_index=False, observed=True)
 .agg(
    types=('type', lambda x: dict(x.value_counts())),
    n_monitored=('type', 'count'),
    rt_collectors=('rt_collectors', lambda x: dict(x.explode().value_counts())),
    urls=('url', np.hstack)
))
variation_distr.sort_values(by=['url_domain', 'n_monitored'], ascending=[True, True], inplace=True)
variation_distr.reset_index(inplace=True, drop=True)

In [37]:
more_than_2_variations = variation_distr[variation_distr['url_domain'] == 'thedopestshop.com']
variation_distr.drop(more_than_2_variations.index, inplace=True)
variation_distr.reset_index(inplace=True, drop=True)
more_than_2_variations

,url_domain,nel_include_subdomains,nel_failure_fraction,nel_success_fraction,nel_max_age,types,n_monitored,rt_collectors,urls
398,thedopestshop.com,false,0.05,0.005,3600,{'other': 1},1,{'nel.heroku.com': 1},[https://thedopestshop.com/apps/ba-loy/app_met...
399,thedopestshop.com,false,1.0,0.0,604800,"{'image': 2, 'css': 1}",3,{'a.nel.cloudflare.com': 3},[https://thedopestshop.com/apps/store-locator/...
400,thedopestshop.com,false,1.0,0.01,604800,"{'script': 108, 'image': 103, 'html': 44, 'css...",268,{'a.nel.cloudflare.com': 268},"[https://thedopestshop.com, https://thedopests..."


In [38]:
variation_distr

,url_domain,nel_include_subdomains,nel_failure_fraction,nel_success_fraction,nel_max_age,types,n_monitored,rt_collectors,urls
0,360chicago.com,false,1.0,0.0,604800,"{'script': 22, '': 1}",23,{'a.nel.cloudflare.com': 23},[https://360chicago.com/cdn-cgi/zaraz/s.js?z=J...
1,360chicago.com,false,1.0,0.01,604800,"{'image': 61, 'script': 33, 'css': 22, 'html':...",145,{'a.nel.cloudflare.com': 145},"[https://360chicago.com, https://360chicago.co..."
2,ablegamers.org,false,1.0,0.0,604800,"{'script': 3, 'font': 3, 'html': 1}",7,{'a.nel.cloudflare.com': 7},[https://ablegamers.org/cdn-cgi/apps/head/Cyua...
3,ablegamers.org,false,1.0,0.01,604800,"{'css': 69, 'script': 53, 'html': 20, 'image':...",150,{'a.nel.cloudflare.com': 150},"[https://ablegamers.org, https://ablegamers.or..."
4,acespace.org,false,1.0,0.01,604800,{'script': 2},2,{'a.nel.cloudflare.com': 2},[https://acespace.org/.hb-bundle/https://cdnjs...
...,...,...,...,...,...,...,...,...,...
475,xxxchurch.com,false,1.0,0.01,604800,"{'image': 107, 'html': 20, 'css': 11, 'script'...",149,{'a.nel.cloudflare.com': 149},"[https://xxxchurch.com, https://xxxchurch.com/..."
476,xyxxcrew.com,false,1.0,0.0,604800,{'other': 2},2,{'a.nel.cloudflare.com': 2},[https://xyxxcrew.com/apps/discountninja/promo...
477,xyxxcrew.com,false,1.0,0.01,604800,"{'image': 901, 'html': 226, 'script': 21, 'oth...",1166,{'a.nel.cloudflare.com': 1166},"[https://xyxxcrew.com, https://xyxxcrew.com/cd..."
478,zishta.com,false,1.0,0.0,604800,"{'script': 1, '': 1}",2,{'a.nel.cloudflare.com': 2},"[https://zishta.com/apps/buckscc/sdk.min.js, h..."


In [41]:
variation_distr[(variation_distr.index % 2 == 0)]

,url_domain,nel_include_subdomains,nel_failure_fraction,nel_success_fraction,nel_max_age,types,n_monitored,rt_collectors,urls
0,360chicago.com,false,1.0,0.0,604800,"{'script': 22, '': 1}",23,{'a.nel.cloudflare.com': 23},[https://360chicago.com/cdn-cgi/zaraz/s.js?z=J...
2,ablegamers.org,false,1.0,0.0,604800,"{'script': 3, 'font': 3, 'html': 1}",7,{'a.nel.cloudflare.com': 7},[https://ablegamers.org/cdn-cgi/apps/head/Cyua...
4,acespace.org,false,1.0,0.01,604800,{'script': 2},2,{'a.nel.cloudflare.com': 2},[https://acespace.org/.hb-bundle/https://cdnjs...
6,acim.org,false,0.05,0.005,3600,"{'script': 3, 'other': 3, 'html': 2, 'css': 2}",10,{'nel.heroku.com': 10},"[https://acim.org/acim, https://acim.org/acim/..."
8,adventuregamers.com,true,0.05,0.0,2592000,"{'html': 14, 'image': 9, 'css': 5}",28,{'nivoli.uriports.com': 28},"[https://adventuregamers.com, https://adventur..."
...,...,...,...,...,...,...,...,...,...
470,wonderrex-online.com,false,1.0,0.0,604800,{'script': 1},1,{'a.nel.cloudflare.com': 1},[https://wonderrex-online.com/apps/giraffly-tr...
472,worthly.com,false,1.0,0.0,604800,{'script': 1},1,{'a.nel.cloudflare.com': 1},[https://worthly.com/cdn-cgi/scripts/5c5dd728/...
474,xxxchurch.com,false,1.0,0.0,604800,{'script': 1},1,{'a.nel.cloudflare.com': 1},[https://xxxchurch.com/cdn-cgi/scripts/5c5dd72...
476,xyxxcrew.com,false,1.0,0.0,604800,{'other': 2},2,{'a.nel.cloudflare.com': 2},[https://xyxxcrew.com/apps/discountninja/promo...


In [42]:
variation_distr[(variation_distr.index % 2 == 1)]

,url_domain,nel_include_subdomains,nel_failure_fraction,nel_success_fraction,nel_max_age,types,n_monitored,rt_collectors,urls
1,360chicago.com,false,1.0,0.01,604800,"{'image': 61, 'script': 33, 'css': 22, 'html':...",145,{'a.nel.cloudflare.com': 145},"[https://360chicago.com, https://360chicago.co..."
3,ablegamers.org,false,1.0,0.01,604800,"{'css': 69, 'script': 53, 'html': 20, 'image':...",150,{'a.nel.cloudflare.com': 150},"[https://ablegamers.org, https://ablegamers.or..."
5,acespace.org,false,1.0,0.0,604800,"{'image': 83, 'other': 40, 'html': 21, 'css': ...",171,{'a.nel.cloudflare.com': 171},"[https://acespace.org, https://acespace.org/wp..."
7,acim.org,false,1.0,0.0,604800,"{'script': 48, 'image': 29, 'css': 23, 'html':...",120,{'a.nel.cloudflare.com': 120},"[https://acim.org, https://acim.org/wp-include..."
9,adventuregamers.com,false,1.0,0.0,604800,"{'': 52, 'script': 37, 'image': 14, 'html': 8,...",123,{'a.nel.cloudflare.com': 123},[https://adventuregamers.com/tardisrocinante/c...
...,...,...,...,...,...,...,...,...,...
471,wonderrex-online.com,false,1.0,0.01,604800,"{'image': 215, 'html': 43, 'script': 11, 'css'...",278,{'a.nel.cloudflare.com': 278},"[https://wonderrex-online.com, https://wonderr..."
473,worthly.com,false,1.0,0.01,604800,"{'image': 127, 'script': 26, 'html': 19, 'css'...",184,{'a.nel.cloudflare.com': 184},"[https://worthly.com, https://worthly.com/wp-c..."
475,xxxchurch.com,false,1.0,0.01,604800,"{'image': 107, 'html': 20, 'css': 11, 'script'...",149,{'a.nel.cloudflare.com': 149},"[https://xxxchurch.com, https://xxxchurch.com/..."
477,xyxxcrew.com,false,1.0,0.01,604800,"{'image': 901, 'html': 226, 'script': 21, 'oth...",1166,{'a.nel.cloudflare.com': 1166},"[https://xyxxcrew.com, https://xyxxcrew.com/cd..."


In [39]:
variation_distr[(variation_distr.index % 2 == 0) & (variation_distr['nel_failure_fraction'] == '0.05')]

,url_domain,nel_include_subdomains,nel_failure_fraction,nel_success_fraction,nel_max_age,types,n_monitored,rt_collectors,urls
6,acim.org,false,0.05,0.005,3600,"{'script': 3, 'other': 3, 'html': 2, 'css': 2}",10,{'nel.heroku.com': 10},"[https://acim.org/acim, https://acim.org/acim/..."
8,adventuregamers.com,true,0.05,0.0,2592000,"{'html': 14, 'image': 9, 'css': 5}",28,{'nivoli.uriports.com': 28},"[https://adventuregamers.com, https://adventur..."
54,cabotcreamery.com,false,0.05,0.005,3600,{'': 1},1,{'nel.heroku.com': 1},[https://cabotcreamery.com/apps/ss]
68,cdllife.com,false,0.05,0.005,3600,"{'other': 48, 'image': 10, 'script': 5, 'html'...",67,{'nel.heroku.com': 67},"[https://cdllife.com/trucking, https://cdllife..."
94,curtsyapp.com,false,0.05,0.005,3600,"{'html': 1, 'css': 1, 'script': 1}",3,{'nel.heroku.com': 3},"[https://curtsyapp.com/help, https://curtsyapp..."
112,doorbraak.be,false,0.05,0.005,3600,"{'script': 80, 'html': 20, 'css': 3, 'font': 3...",109,{'nel.heroku.com': 109},"[https://doorbraak.be, https://doorbraak.be/nu..."
152,getschooled.com,false,0.05,0.005,3600,{'other': 1},1,{'nel.heroku.com': 1},[https://getschooled.com/v2/graphql]
168,hexclad.com,false,0.05,0.005,3600,{'other': 1},1,{'nel.heroku.com': 1},[https://hexclad.com/apps/ba-loy/app_metrics]
206,landing.jobs,false,0.05,0.005,3600,"{'image': 33, 'html': 8, 'other': 3, 'css': 3,...",50,{'nel.heroku.com': 50},"[https://landing.jobs/ahoy/visits, https://lan..."
242,mintvelvet.com,false,0.05,0.005,3600,{'html': 1},1,{'nel.heroku.com': 1},[https://mintvelvet.com/apps/attraqt]


In [40]:
variation_distr[(variation_distr.index % 2 == 1) & (variation_distr['nel_failure_fraction'] == '0.05')]

,url_domain,nel_include_subdomains,nel_failure_fraction,nel_success_fraction,nel_max_age,types,n_monitored,rt_collectors,urls
19,anytrivia.com,false,0.05,0.005,3600,"{'script': 19, 'html': 14, 'image': 10, 'css':...",48,{'nel.heroku.com': 48},"[https://anytrivia.com, https://anytrivia.com/..."
33,be-salt.com,false,0.05,0.005,3600,"{'script': 24, 'image': 18, 'html': 7, 'other'...",62,{'nel.heroku.com': 62},"[https://be-salt.com, https://be-salt.com/_nex..."
37,bemz.com,false,0.05,0.005,3600,"{'script': 22, 'html': 20, 'css': 8, 'image': ...",63,{'nel.heroku.com': 63},"[https://bemz.com, https://bemz.com/_next/stat..."
39,benable.com,false,0.05,0.005,3600,"{'script': 121, 'css': 59, 'image': 38, 'html'...",238,{'nel.heroku.com': 238},"[https://benable.com, https://benable.com/_app..."
43,biomassmagazine.com,false,0.05,0.005,3600,"{'script': 44, '': 21, 'other': 21, 'html': 20...",121,{'nel.heroku.com': 121},"[https://biomassmagazine.com, https://biomassm..."
49,border.co.jp,false,0.05,0.005,3600,"{'image': 187, 'html': 20, 'script': 17, 'css'...",236,{'nel.heroku.com': 236},"[https://border.co.jp, https://border.co.jp/wp..."
51,borsod24.hu,false,0.05,0.005,3600,"{'html': 20, 'script': 7, 'css': 6, 'font': 1}",34,{'nel.heroku.com': 34},"[https://borsod24.hu, https://borsod24.hu/app/..."
79,cloudsecurityalliance.org,false,0.05,0.005,3600,"{'image': 99, 'html': 83, 'other': 2, 'css': 1...",187,{'nel.heroku.com': 187},"[https://cloudsecurityalliance.org, https://cl..."
81,coinmap.org,false,0.05,0.005,3600,"{'other': 27, 'image': 23, 'html': 20, 'css': ...",91,{'nel.heroku.com': 91},"[https://coinmap.org, https://coinmap.org/cdn/..."
85,commonwealth.im,false,0.05,0.005,3600,"{'script': 3, 'html': 1}",4,{'nel.heroku.com': 4},"[https://commonwealth.im, https://commonwealth..."
